## Pooling for Sequencing

### Converting a confusing spreadsheet to an automated pooling protocol

### Goals:

*  Create a script that will calculate how to evenly or unevenly pool samples for different sequencers

* Provide drop down menu of sequencer options for user to chose from (all have different total read capacities and loading requirements, so selection must then be linked to those details in some way -- library/dictionary/ect)

* Give total concentration of that final library

* Asks if user wants phix and what percentage, then calculate how many reads go to phix, and how many leftover for samples

* Asks ask how many libraries going on

* Asks for number of samples per library, number of reads per sample, ng/ul, % of library between 200-1000 base pair length, base pair length average

* From input, will calculate MW, nmol/ul, nM, total number of reads needed, and proportion of the lib/total

* Will ask user what minimum pipetting value they want to provide

* After the user enters in all libraries and their respective information the script will calculate volume to input (uL) but if that volume is under the minimum pipetting value, will recalculate so proportionally input is greater than that minimum.

* Will also provide instructions on barcoding directions depending on sequencer

* Finally, will calculate dilutions or total volume to provide per sequencer requirements

In [10]:
import tkinter as tk
import Bio as BP
import pandas as pd
import numpy as np
import collections as col
import os
import csv
import re
import regex

In [11]:
'''Resource for tkinter GUI (graphical user interface)
http://web.archive.org/web/20201111235321/https://effbot.org/tkinterbook/tkinter-hello-tkinter.htm
'''
def pick_sequencer():
    
    OPTIONS = [
    "NovaSeq S4 4 Lanes (8 billion clusters $30,780)",
    "NovaSeq S4 3 Lanes (6 billion clusters $25,335)",
    "NovaSeq S4 2 Lanes (4 billion clusters $16,890)",
    "NovaSeq S4 1 Lane (2 billion clusters $8445)",
    "NovaSeq S4 4 Lanes (8 billion clusters $30,780)",
    "NovaSeq S4 3 Lanes (6 billion clusters $25,335)",
    "NovaSeq S4 2 Lanes (4 billion clusters $16,890)",
    "NovaSeq S4 1 Lane (2 billion clusters $8445)",
    "NovaSeq S2 2 Lanes (3 billion clusters $10,575 - $16,700)",
    "NovaSeq S2 1 Lane (1.5 billion clusters $6687 - $9750)",
    "NovaSeq S2 2 Lanes (3 billion clusters $10,575 - $16,700)",
    "NovaSeq S2 1 Lane (1.5 billion clusters $6687 - $9750)",
    "NovaSeq SP (800 million clusters $2250 - $5500)",
    "NextSeq 2000 P3 (1.1 billion clusters $2762 - $5100)",
    "NextSeq 2000 P3 (1.1 billion clusters $2762 - $5100)",
    "NextSeq High Output (400 million clusters $1525 - $4680)",
    "NextSeq High Output (400 million clusters $1525 - $4680)",
    "NextSeq High Output (400 million clusters $1525 - $4680)",
    "NextSeq Mid Output (150 million clusters $1110 - $1790)",
    "NextSeq Mid Output (150 million clusters $1110 - $1790)",
    "MiSeq V3 (23 million clusters $930 - $1625)",
    "MiSeq V3 (23 million clusters $930 - $1625)",
    "MiSeq V2 (13 million clusters $840 - $1210)",
    "MiSeq V2 (13 million clusters $840 - $1210)",
    "MiSeq V2 (13 million clusters $840 - $1210)",
    "MiSeq V2 Micro (4 million clusters $450)",
    "MiSeq V2 Nano (1 million clusters $300 - $360)",
    "MiSeq V2 Nano (1 million clusters $300 - $360)",
    "iSeq (4 million clusters $625)",
    ] 

    Sequencer = tk.Tk()

    variable = tk.StringVar(Sequencer)
    variable.set(OPTIONS[0]) # default value

    m = tk.Label(Sequencer, text="Please pick your sequencer!")
    w = tk.OptionMenu(Sequencer, variable, *OPTIONS)
    m.pack()
    w.pack()


    picked = []
    Close = tk.Label(Sequencer, text="Great! Now please click the X to close.")
    
    def ok():
        stuff = variable.get()
        picked.append(str(stuff))
        Close.pack()
        ### why isn't this returning? I know it's a scope issue, but I can't save output as 
        ###variable the way that I'm used to with that button command below :( 
        
    button = tk.Button(Sequencer, text="Submit", command=ok)
    button.pack()


    #def close():
        #Sequencer.destroy()

    #secondbutton = tk.Button(Sequencer, text ="Close Window", command=close)
    #secondbutton.pack()    
    #I never figured out why this doesn't work :( 
    
    Sequencer.mainloop()
    return picked[0]
    
   

In [12]:
def sequencer_param(sequencer):
    if sequencer == "iSeq (4 million clusters $625)":
        volume_input_uL = 10 
        concentration_nM = range(4, 20)     
        
    if "MiSeq" in sequencer:
        volume_input_uL = 10 
        concentration_nM = range(4, 20)   
    if "NextSeq" in sequencer:
        volume_input_uL = 10 
        concentration_nM = range(4, 20)     
        
    if "NovaSeq SP" in sequencer:
        volume_input_uL = 100 
        concentration_nM = range(4, 10)  
    
    if "NovaSeq S2" in sequencer:
        volume_input_uL = 200 
        concentration_nM = range(4, 10)   
    
    if "NovaSeq S4" in sequencer:
        volume_input_uL = 350 
        concentration_nM = range(4, 10)  
        
    return volume_input_uL, concentration_nM

In [33]:
def library_number_and_names():
    lib_num = input("How many libraries are you sequencing? ")
    lib_num = int(lib_num) 
    lib_names = {}
    for i in range(lib_num):
        question = "What is the name of library number " + str(i) + "? "
        lib_names[i] = input(question)
    return lib_num, lib_names

### this needs to be a dictionary to prevent people using same name twice



In [34]:
def samples_per_lib_and_reads_per_sample(lib_num, lib_names):
    dictionary_of_libraries = {}
    for name in lib_names:
        input_text = "How many samples in " + str(name) + "? "
        dictionary_of_libraries[name] = input(input_text)
        
        
    return dictionary_of_libraries
    
    ### I think I've reached the need to understand
    ### dictionaries better than I do 
    ### I want the library name to be key to number of samples maybe?
    ### perhaps I will add the other metrics for that library as a list 
    ### that's attached to a key in a dictionary? or should I use df???
        


In [35]:
def main():
    print("Hello user! This script will help you pool your libraries for sequencing. Just follow the instructions on the screen.")
    sequencer = pick_sequencer()
    volume_input_uL, concentration_nM = sequencer_param(sequencer)
    lib_num, lib_names = library_number_and_names()
    dictionary_of_libraries = samples_per_lib_and_reads_per_sample(lib_num, lib_names)
    

    print(sequencer, volume_input_uL, concentration_nM, lib_num, lib_names, dictionary_of_libraries)

In [36]:
main()

Hello user! This script will help you pool your libraries for sequencing. Just follow the instructions on the screen.
How many libraries are you sequencing? 5
What is the name of library number 0? bob
What is the name of library number 1? joe
What is the name of library number 2? joe
What is the name of library number 3? joe
What is the name of library number 4? joe
How many samples in 0? 3
How many samples in 1? 4
How many samples in 2? 5
How many samples in 3? 3
How many samples in 4? 1
NovaSeq S4 4 Lanes (8 billion clusters $30,780) 350 range(4, 10) 5 {0: 'bob', 1: 'joe', 2: 'joe', 3: 'joe', 4: 'joe'} {0: '3', 1: '4', 2: '5', 3: '3', 4: '1'}
